# Google Colab Rust Setup

The following cell is used to set up and spin up a Jupyter Notebook environment with a Rust kernel using Nix and IPC Proxy. 

In [ ]:
!wget -qO- https://gist.github.com/wiseaidev/2af6bef753d48565d11bcd478728c979/archive/3f6df40db09f3517ade41997b541b81f0976c12e.tar.gz | tar xvz --strip-components=1
!bash setup_evcxr_kernel.sh

## Memory Management and Pointers

### Challenges of Manual Memory Management

#### Ownership and Borrowing

In [2]:
let s1 = String::from("hello");
let s2 = s1;
// println!("s1: {}", s1);
println!("s2: {}", s2);

s2: hello


In [3]:
let s1 = String::from("hello");
let s2 = s1;
println!("s1: {}", s1);
println!("s2: {}", s2);

Error: borrow of moved value: `s1`

#### Dangling References

In [6]:
fn main() {
    let r;
    {
        let x = 42;
        r = &x;
    }
    println!("r: {}", r);
}

main()

Error: `x` does not live long enough

#### Data Races

In [7]:
use std::thread;

let mut data = vec![1, 2, 3];

let handle1 = thread::spawn(move || {
    data.push(4);
});

handle1.join().unwrap();

println!("{:?}", data);

Error: variable `r` is assigned to, but never used

Error: value assigned to `r` is never read

Error: borrow of moved value: `data`

#### Unsafe Code

In [8]:
let mut numbers = vec![1, 2, 3, 4, 5];

let mut_ptr = numbers.as_mut_ptr();

let len = numbers.len();

unsafe {
    for i in 0..len {
       let current_value = *mut_ptr.add(i);
        *mut_ptr.add(i) = current_value * 2;
    }
}

println!("{:?}", numbers);

[2, 4, 6, 8, 10]


#### Resource Management

In [10]:
use std::fs::File;

let file = File::create("example.txt")?;

// File will be automatically closed when it goes out of scope

// Additional code for working with the file goes here

### Stack vs. Heap

#### The Stack's Role in Rust

In [11]:
fn main() {
    let x = 42;
    println!("The answer is: {}", x);
}

main()

The answer is: 42


()

#### The Heap's Role in Rust

In [12]:
fn main() {
    let s1 = String::from("Hello");
    let s2 = s1;
    println!("{}", s2);
}

main()

Hello


()

#### The Ownership Model

In [13]:
fn main() {
    let s = String::from("Rust");
    println!("{}", s);
}

main()

Rust


()

#### Borrowing in Rust

In [14]:
fn calculate_length(s: &String) -> usize {
    s.len()
}

let s = String::from("Rust");
let len = calculate_length(&s);
println!("Length of '{}' is {}.", s, len);

Length of 'Rust' is 4.


#### Lifetimes in Rust

In [24]:
:dep thiserror = {version = "1.0.50"}

In [15]:
fn get_length(s: &String) -> usize {
    s.len()
}

let result;
{
    let s = String::from("Rust");
    result = get_length(&s);
}

println!("Length: {}", result);

Length: 4


### Pointers & Smart Pointers

#### Box

In [17]:
fn main() {
    let complex_data = Box::new(vec![1, 2, 3, 4, 5]);

    let sum: i32 = complex_data.iter().sum();

    println!("The sum of the vector is: {}", sum);
}

main()

The sum of the vector is: 15


()

#### Rc

In [18]:
use std::rc::Rc;

let data = Rc::new(vec![1, 2, 3, 4, 5]);

let clone1 = Rc::clone(&data);
let clone2 = Rc::clone(&data);

let sum: i32 = data.iter().sum();

println!("Data: {:?}", data);
println!("Clone1: {:?}", clone1);
println!("Clone2: {:?}", clone2);

Data: [1, 2, 3, 4, 5]
Clone1: [1, 2, 3, 4, 5]
Clone2: [1, 2, 3, 4, 5]


#### Arc

In [19]:
use std::sync::Arc;
use std::thread;

let data = Arc::new(vec![1, 2, 3, 4, 5]);

let clone1 = Arc::clone(&data);
let clone2 = Arc::clone(&data);

let handle1 = thread::spawn(move || {
    let sum: i32 = clone1.iter().sum();
    println!("Thread 1 - Sum of Data: {}", sum);
});

let handle2 = thread::spawn(move || {
    let product: i32 = clone2.iter().product();
    println!("Thread 2 - Product of Data: {}", product);
});

handle1.join().unwrap();
handle2.join().unwrap();

Thread 1 - Sum of Data: 15
Thread 2 - Product of Data: 120


#### RefCell

In [5]:
use std::cell::RefCell;

fn main() {
    let data = RefCell::new(vec![1, 2, 3]);

    let data_ref = data.borrow();

    // data_ref.push(4);

    drop(data_ref);

    let mut data_ref_mut = data.borrow_mut();
    data_ref_mut.push(4);

    println!("Modified Data: {:?}", *data_ref_mut);
}

main()

Modified Data: [1, 2, 3, 4]


()

In [4]:
use std::cell::RefCell;

let data = RefCell::new(vec![1, 2, 3]);

let data_ref = data.borrow();

data_ref.push(4);

drop(data_ref);

let mut data_ref_mut = data.borrow_mut();
data_ref_mut.push(4);

println!("Modified Data: {:?}", *data_ref_mut);

Error: cannot borrow data in dereference of `Ref<'_, Vec<i32>>` as mutable

Error: The variable `data_ref_mut` contains a reference with a non-static lifetime so
can't be persisted. You can prevent this error by making sure that the
variable goes out of scope - i.e. wrapping the code in {}.

#### Mutex

In [23]:
use std::sync::{Mutex, Arc};
use std::thread;
use std::time::Duration;
use std::collections::HashMap;

struct Bank {
    accounts: Mutex<HashMap<String, f64>>,
}

impl Bank {
    fn new() -> Self {
        Bank {
            accounts: Mutex::new(HashMap::new()),
        }
    }

    fn deposit(&self, account: &str, amount: f64) {
        let mut accounts = self.accounts.lock().unwrap();
        let balance = accounts.entry(account.to_string()).or_insert(0.0);
        *balance += amount;
    }

    fn withdraw(&self, account: &str, amount: f64) {
        let mut accounts = self.accounts.lock().unwrap();
        let balance = accounts.entry(account.to_string()).or_insert(0.0);
        if *balance >= amount {
            *balance -= amount;
        } else {
            println!("Insufficient funds for account: {}", account);
        }
    }

    fn check_balance(&self, account: &str) -> f64 {
        let accounts = self.accounts.lock().unwrap();
        *accounts.get(account).unwrap_or(&0.0)
    }
}

let bank = Arc::new(Bank::new());
let mut handles = vec![];

for i in 0..5 {
    let bank_clone = Arc::clone(&bank);
    let handle = thread::spawn(move || {
        let account = format!("Account{}", i);
        bank_clone.deposit(&account, 100.0);
        thread::sleep(Duration::from_millis(100)); // Simulate other work
        bank_clone.withdraw(&account, 30.0);
        let balance = bank_clone.check_balance(&account);
        println!("{} - Balance: {:.2}", account, balance);
    });
    handles.push(handle);
}

for handle in handles {
    handle.join().unwrap();
}

Account1 - Balance: 70.00
Account0 - Balance: 70.00
Account3 - Balance: 70.00
Account4 - Balance: 70.00
Account2 - Balance: 70.00


()

#### RwLock

In [29]:
use std::sync::{Arc, RwLock};
use std::thread;
use std::time::Duration;
use std::collections::HashMap;

struct Bank {
    accounts: RwLock<HashMap<String, f64>>,
}

impl Bank {
    fn new() -> Self {
        Bank {
            accounts: RwLock::new(HashMap::new()),
        }
    }

    fn deposit(&self, account: &str, amount: f64) {
        let mut accounts = self.accounts.write().unwrap();
        let balance = accounts.entry(account.to_string()).or_insert(0.0);
        *balance += amount;
    }

    fn withdraw(&self, account: &str, amount: f64) {
        let mut accounts = self.accounts.write().unwrap();
        let balance = accounts.entry(account.to_string()).or_insert(0.0);
        if *balance >= amount {
            *balance -= amount;
        } else {
            println!("Insufficient funds for account: {}", account);
        }
    }

    fn check_balance(&self, account: &str) -> f64 {
        let accounts = self.accounts.read().unwrap();
        *accounts.get(account).unwrap_or(&0.0)
    }
}

let bank = Arc::new(Bank::new());
let mut handles = vec![];

for i in 0..5 {
    let bank_clone = Arc::clone(&bank);
    let handle = thread::spawn(move || {
        let account = format!("Account{}", i);
        bank_clone.deposit(&account, 100.0);
        thread::sleep(Duration::from_millis(100)); // Simulate other work
        bank_clone.withdraw(&account, 30.0);
        let balance = bank_clone.check_balance(&account);
        println!("{} - Balance: {:.2}", account, balance);
    });
    handles.push(handle);
}

for handle in handles {
    handle.join().unwrap();
}

// TODO: investigate why it throws a seg fault once.

Account1 - Balance: 70.00
Account0 - Balance: 70.00
Account2 - Balance: 70.00
Account3 - Balance: 70.00
Account4 - Balance: 70.00


()

#### Atomic

In [30]:
use std::sync::atomic::{AtomicBool, Ordering};
use std::sync::Arc;
use std::thread;

let atomic_flag = Arc::new(AtomicBool::new(false));

let atomic_flag_clone = Arc::clone(&atomic_flag);

let thread_handle = thread::spawn(move || {
    thread::sleep(std::time::Duration::from_secs(1));
    atomic_flag_clone.store(true, Ordering::Relaxed);
});

while !atomic_flag.load(Ordering::Relaxed) {
    // Do some work...
}

thread_handle.join().unwrap();

println!("Atomic flag is set to true.");

Atomic flag is set to true.


### Unsafe Rust

#### Unsafe Functions

In [31]:
let mut x = 42;
let raw_ptr: *const i32 = &x;

unsafe {
    println!("The value at raw_ptr: {}", *raw_ptr);
}

The value at raw_ptr: 42


()

#### Unsafe Traits

In [32]:
struct MyBox<T>(T);

impl<T> MyBox<T> {
    fn new(x: T) -> MyBox<T> {
        MyBox(x)
    }
}

use std::ops::Deref;

impl<T> Deref for MyBox<T> {
    type Target = T;

    fn deref(&self) -> &T {
        &self.0
    }
}

let my_box = MyBox::new(42);
*my_box

42

#### Custom Unsafe Abstractions

In [33]:
mod custom_device {
    pub struct HardwareDevice {
        // Fields specific to the hardware device
    }

    impl HardwareDevice {
        pub fn new() -> Self {
            // Initialize and configure the hardware device
            Self {
                // Initialize fields here
            }
        }

        pub fn perform_unsafe_operation(&self) {
            // Perform low-level operations that require `unsafe` Rust
        }
    }
}

let device = custom_device::HardwareDevice::new();

// Interact with the hardware device safely
device.perform_unsafe_operation();

#### Lifetime Annotations

In [38]:
fn longest<'a>(s1: &'a str, s2: &'a str) -> &'a str {
    if s1.len() > s2.len() {
        s1
    } else {
        s2
    }
}

let s1 = "Hello,";
let s2 = "world!";

let longest_string = longest(s1, s2);

println!("The longest string is: {}", longest_string);

The longest string is: world!


#### Custom Memory Allocators

In [40]:
use std::alloc::{GlobalAlloc, Layout};
use std::ptr;

struct MyAllocator;

unsafe impl GlobalAlloc for MyAllocator {
    unsafe fn alloc(&self, layout: Layout) -> *mut u8 {
        // Implement custom allocation logic here
        // Return a pointer to the allocated memory
        let size = layout.size();
        if size == 0 {
            return ptr::null_mut();
        }

        // For demonstration purposes, we'll use a simple global memory buffer
        // This is just an example and not suitable for production use
        static mut MEMORY: [u8; 4096] = [0; 4096];
        static mut NEXT: usize = 0;

        let align = layout.align();
        let aligned_next = (NEXT + align - 1) & !(align - 1);
        if aligned_next + size <= MEMORY.len() {
            NEXT = aligned_next + size;
            &mut MEMORY[aligned_next] as *mut u8
        } else {
            ptr::null_mut()
        }
    }

    unsafe fn dealloc(&self, _ptr: *mut u8, _layout: Layout) {
        // Implement custom deallocation logic here
        // This allocator does not support deallocation in this example
    }
}

#[global_allocator]
static GLOBAL_ALLOCATOR: MyAllocator = MyAllocator;

// Allocate memory using our custom allocator
let size = 64;
let layout = Layout::from_size_align(size, 8).unwrap();
let ptr = unsafe { GLOBAL_ALLOCATOR.alloc(layout) };

if !ptr.is_null() {
    println!("Allocated memory at {:?}", ptr);
} else {
    println!("Allocation failed");
}

// Deallocate memory (not supported in this example)

Allocated memory at 0x7f3742c55011


()

---
---